In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
tf.__version__

'2.4.0'

In [6]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [7]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [8]:
print(y)

[1 0 1 ... 1 1 0]


Label Encoding Categorical Gender data


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [11]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the 'Geography' column

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [15]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting the dataset into test set and training set

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

Feature Scaling

In [26]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

### PART 2 - BUILDING THE ANN

Initializing the ANN

In [27]:
ann = tf.keras.models.Sequential()

Adding the input layer and the first hidden layer

In [28]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the second hidden layer

In [29]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the output layer

In [30]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# activation for non_binary output for predicting more than two or categories

### PART 3 - TRAINING THE ANN

Compiling the ANN

In [31]:
ann.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])
# categorical_crossentropy for non_binary output

Training the ANN on the Training set

In [32]:
ann.fit(x_train, y_train, batch_size= 32, epochs=100)
# default classic value for batch_size = 32

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6238 - accuracy: 0.7295
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4829 - accuracy: 0.7962
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4421 - accuracy: 0.8020
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4262 - accuracy: 0.8123
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4233 - accuracy: 0.8211
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4209 - accuracy: 0.8218
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4091 - accuracy: 0.8257
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4000 - accuracy: 0.8281
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4003 - accuracy: 0.8319
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3971 - accura

### Part 4 - Making the predictions and evaluating the model

Predicting the result of a single observation

In [37]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))>0.5)

[[False]]


Predicting the test set results

In [38]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making the Confusion Matrix

In [39]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1503   92]
 [ 193  212]]


0.8575